<a href="https://colab.research.google.com/github/osawa-fernando/ML/blob/main/Translate_and_Create_Subtitle_(Source_Youtube_Language_PT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yt-dlp moviepy openai-whisper ffmpeg-python deep-translator
!sudo apt update && sudo apt install ffmpeg -y

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 21.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.2 MB/s e

Download YT Video

In [2]:
import yt_dlp

# 🔗 Cole o link do YouTube aqui:
video_url = "https://www.youtube.com/watch?v=1VVXd6YK3Qc"  # ⬅️ TROQUE AQUI

# Configurações de download
ydl_opts = {
    'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
    'outtmpl': 'video_original.%(ext)s',
    'merge_output_format': 'mp4'
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

[youtube] Extracting URL: https://www.youtube.com/watch?v=1VVXd6YK3Qc
[youtube] 1VVXd6YK3Qc: Downloading webpage
[youtube] 1VVXd6YK3Qc: Downloading tv client config
[youtube] 1VVXd6YK3Qc: Downloading player cbd59245-main
[youtube] 1VVXd6YK3Qc: Downloading tv player API JSON
[youtube] 1VVXd6YK3Qc: Downloading ios player API JSON
[youtube] 1VVXd6YK3Qc: Downloading m3u8 information
[info] Testing format 616
[info] 1VVXd6YK3Qc: Downloading 1 format(s): 616+140-9
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 69
[download] Destination: video_original.f616.mp4
[download] 100% of  108.18MiB in 00:00:04 at 23.91MiB/s                
[download] Destination: video_original.f140-9.m4a
[download] 100% of    5.99MiB in 00:00:00 at 30.16MiB/s  
[Merger] Merging formats into "video_original.mp4"
Deleting original file video_original.f140-9.m4a (pass -k to keep)
Deleting original file video_original.f616.mp4 (pass -k to keep)


Extract audio for transcription (.wav for better results)

In [3]:
from moviepy.editor import VideoFileClip

clip = VideoFileClip("video_original.mp4")
clip.audio.write_audiofile("audio.wav")

  if event.key is 'enter':



MoviePy - Writing audio in audio.wav


MoviePy - Done.


Transcription (w/ Whisper)

In [4]:
import whisper

model = whisper.load_model("base")  # Opções: tiny, base, small, medium, large
result = model.transcribe("audio.wav", task="transcribe")
segments = result["segments"]

100%|███████████████████████████████████████| 139M/139M [00:05<00:00, 25.8MiB/s]


Translate to Portuguese (Google Translate)

In [5]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='pt')

for seg in segments:
    seg['text_pt'] = translator.translate(seg['text'])

Gerar arquivo .srt de legenda

In [6]:
def format_srt_timestamp(seconds):
    from datetime import timedelta
    t = str(timedelta(seconds=seconds))
    if '.' in t:
        t, ms = t.split('.')
    else:
        ms = "000"
    return f"{t.zfill(8).replace('.', ',')},{ms[:3]}"

with open("legenda_traduzida.srt", "w", encoding="utf-8") as f:
    for i, seg in enumerate(segments, 1):
        start = format_srt_timestamp(seg['start'])
        end = format_srt_timestamp(seg['end'])
        text = seg['text_pt']
        f.write(f"{i}\n{start} --> {end}\n{text}\n\n")

Embutir a legenda

In [7]:
# Gera vídeo final com as legendas embutidas
output_path = "video_legendado.mp4"

!ffmpeg -i video_original.mp4 -vf "subtitles=legenda_traduzida.srt:force_style='FontName=Arial,FontSize=18'" -c:a copy {output_path}

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Download .mp4 w/ subtitles

In [8]:
from google.colab import files
files.download("video_legendado.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>